In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, RNN, LSTMCell, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

def load_data(dataset_path, img_size=(192, 192)):
    """
    Loads precomputed mel-spectrogram images and extracts labels from folder names.
    Resizes images to 192x192 for higher resolution.
    """
    X = []
    y = []
    class_names = sorted(os.listdir(dataset_path))  # Get emotion categories
    
    for label in class_names:
        class_path = os.path.join(dataset_path, label)
        if not os.path.isdir(class_path):
            continue
        
        files = glob.glob(os.path.join(class_path, '*.png'))
        print(f"Found {len(files)} images for class '{label}'.")
        
        for file in files:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
            img = cv2.resize(img, img_size)  # Resize to higher resolution
            X.append(img)
            y.append(label)
    
    X = np.array(X, dtype=np.float32) / 255.0  # Normalize pixel values
    y = np.array(y)
    return X, y, class_names

def augment_data(X, y, augmentation_factor=0.3):
    """Apply simple data augmentation to improve model generalization"""
    aug_X = []
    aug_y = []
    
    for i in range(len(X)):
        # Original sample
        aug_X.append(X[i])
        aug_y.append(y[i])
        
        # Apply time stretching (horizontal scaling)
        if np.random.random() < augmentation_factor:
            stretched = cv2.resize(X[i], (int(X[i].shape[1] * 1.1), X[i].shape[0]))
            stretched = cv2.resize(stretched, (X[i].shape[1], X[i].shape[0]))
            aug_X.append(stretched)
            aug_y.append(y[i])
        
        # Apply frequency masking (random horizontal lines masked)
        if np.random.random() < augmentation_factor:
            masked = X[i].copy()
            num_masks = np.random.randint(1, 4)
            for _ in range(num_masks):
                freq_width = np.random.randint(5, 20)  # Wider masks for larger image
                freq_start = np.random.randint(0, X[i].shape[0] - freq_width)
                masked[freq_start:freq_start+freq_width, :] = 0
            aug_X.append(masked)
            aug_y.append(y[i])
    
    return np.array(aug_X), np.array(aug_y)

def preprocess_labels(y):
    """Encodes string labels into numerical one-hot vectors."""
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_onehot = to_categorical(y_encoded)
    return y_onehot, le

def build_hybrid_model_192(input_shape, num_classes):
    """
    Builds a hybrid CNN-LSTM model for 192x192 inputs.
    Adjusted architecture to handle larger input dimensions.
    """
    model = Sequential()
    
    # First, add CNN layers to extract spatial features
    model.add(Reshape((input_shape[0], input_shape[1], 1), input_shape=input_shape))
    
    # First conv block
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))  # Output: 96x96x32
    
    # Second conv block
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))  # Output: 48x48x64
    
    # Third conv block - added for larger input size
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))  # Output: 24x24x128
    
    # Reshape for LSTM - treat rows as time steps
    # After 3 MaxPooling layers, we have 24x24x128, reshape to 24 timesteps with 24*128 features
    model.add(Reshape((24, 24*128)))
    
    # LSTM layers for temporal features
    model.add(RNN(LSTMCell(256), return_sequences=True))  # Increased units for larger input
    model.add(Dropout(0.3))
    model.add(RNN(LSTMCell(128)))  # Increased units
    model.add(Dropout(0.3))
    
    # Classification layers
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))  # Increased units
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

def main():
    # Configuration parameters
    dataset_path = 'RAVDESS_mel_spectrograms'  # Update with your dataset path
    img_size = (192, 192)  # Increased from 128x128 to 192x192
    test_size = 0.15
    val_size = 0.15
    batch_size = 16
    max_epochs = 100
    
    # Load and preprocess data with larger image size
    X, y, class_names = load_data(dataset_path, img_size)
    print(f"Original dataset: {X.shape[0]} samples")
    print(f"Image shape: {X[0].shape}")
    
    # Apply data augmentation
    X_aug, y_aug = augment_data(X, y)
    print(f"Augmented dataset: {X_aug.shape[0]} samples")
    
    # Encode labels
    y_onehot, le = preprocess_labels(y_aug)
    
    # Create train, validation, and test sets
    # First split off the test set
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_aug, y_onehot, test_size=test_size, random_state=42, 
        stratify=np.argmax(y_onehot, axis=1)
    )
    
    # Then split the remaining data into train and validation
    val_ratio = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_ratio, random_state=42,
        stratify=np.argmax(y_temp, axis=1)
    )
    
    print(f"Train: {X_train.shape[0]}, Validation: {X_val.shape[0]}, Test: {X_test.shape[0]}")
    
    # Build and compile the model for 192x192 inputs
    input_shape = X_train.shape[1:]  # (192, 192)
    num_classes = y_onehot.shape[1]
    model = build_hybrid_model_192(input_shape, num_classes)
    
    # Memory usage estimate
    param_count = model.count_params()
    print(f"Model has {param_count:,} parameters")
    
    # Create optimizer with learning rate
    initial_lr = 0.001
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    
    # Print model summary
    model.summary()
    
    # Callbacks for training
    checkpoint = ModelCheckpoint(
        'best_emotion_model_192x192.h5',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=1
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=max_epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stop, checkpoint, reduce_lr],
        verbose=1
    )
    
    # Evaluate on the test set
    test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    
    # Generate predictions and evaluate in detail
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(
        y_true_classes, 
        y_pred_classes, 
        target_names=[le.inverse_transform([i])[0] for i in range(num_classes)]
    ))
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='lower right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history_192x192.png')
    plt.show()

if __name__ == '__main__':
    main()

Found 192 images for class 'angry'.
Found 192 images for class 'calm'.
Found 192 images for class 'disgust'.
Found 192 images for class 'fearful'.
Found 192 images for class 'happy'.
Found 96 images for class 'neutral'.
Found 192 images for class 'sad'.
Found 192 images for class 'surprised'.
Original dataset: 1440 samples
Image shape: (192, 192)
Augmented dataset: 2352 samples
Train: 1646, Validation: 353, Test: 353
Model has 3,772,200 parameters
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 192, 192, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 192, 192, 32)     128       
 ormalization)                                                   


KeyboardInterrupt: 